In [17]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import csv
import re
import requests
import glob
import os
import time


In [18]:
# Dictionary for the raw data, the key is the wbesite containing the raw data and the key is the ID relating to the table required on the web page

def league_stats(seasons):

    keywords_1 = ['stats', 'keepers', 'shooting', 'misc', 'passing', 'defense']
    lst_of_dicts = []

    for season in seasons:
        for keyword in keywords_1:

            if keyword=='stats':
                team_id = {'id': {'stats_squads_standard_for'}}
                player_id = {'id':{'stats_standard'}}

            elif keyword=='keepers':
                team_id = {'id': {'stats_squads_keeper_for'}}
                player_id = {'id':{'stats_keeper'}}

            elif keyword=='shooting':
                team_id = {'id': {'stats_squads_shooting_for'}}
                player_id = {'id':{'stats_shooting'}}

            elif keyword=='misc':
                team_id = {'id': {'stats_squads_misc_for'}}
                player_id = {'id':{'stats_misc'}}

            elif keyword=='passing':
                team_id = {'id': {'stats_squads_passing_for'}}
                player_id = {'id':{'stats_passing'}}

            elif keyword=='defense':
                team_id = {'id': {'stats_squads_defense_for'}}
                player_id = {'id':{'stats_defense'}}

            else:
                None

            if keyword in ('passing','defense'):
                spa_urls = f'https://fbref.com/en/comps/12/{season}/{keyword}/{season}-La-Liga-Stats'
                fra_urls = f'https://fbref.com/en/comps/13/{season}/{keyword}/{season}-Ligue-1-Stats'
                eng_urls = f'https://fbref.com/en/comps/9/{season}/{keyword}/{season}-Premier-League-Stats'
                ger_urls = f'https://fbref.com/en/comps/20/{season}/{keyword}/{season}-Bundesliga-Stats'
                por_urls = f'https://fbref.com/en/comps/32/{season}/{keyword}/{season}-Primeira-Liga-Stats'
                ita_urls = f'https://fbref.com/en/comps/11/{season}/{keyword}/{season}-Serie-A-Stats'


                dict = {spa_urls: [team_id,player_id], fra_urls: [team_id,player_id], eng_urls: [team_id,player_id], 
                          ger_urls: [team_id,player_id], por_urls: [team_id,player_id], ita_urls: [team_id,player_id]
                        }
  
                
                lst_of_dicts.append(dict)

            else:
                spa_urls = f'https://fbref.com/en/comps/12/{season}/{keyword}/{season}-La-Liga-Stats'
                fra_urls = f'https://fbref.com/en/comps/13/{season}/{keyword}/{season}-Ligue-1-Stats'
                eng_urls = f'https://fbref.com/en/comps/9/{season}/{keyword}/{season}-Premier-League-Stats'
                ger_urls = f'https://fbref.com/en/comps/20/{season}/{keyword}/{season}-Bundesliga-Stats'
                por_urls = f'https://fbref.com/en/comps/32/{season}/{keyword}/{season}-Primeira-Liga-Stats'
                ita_urls = f'https://fbref.com/en/comps/11/{season}/{keyword}/{season}-Serie-A-Stats'
                bul_urls = f'https://fbref.com/en/comps/67/{season}/{keyword}/{season}-Bulgarian-First-League-Stats'
                aus_urls = f'https://fbref.com/en/comps/56/{season}/{keyword}/{season}-Austrian-Bundesliga-Stats'
                dan_urls = f'https://fbref.com/en/comps/50/{season}/{keyword}/{season}-Danish-Superliga-Stats'
                bel_urls = f'https://fbref.com/en/comps/37/{season}/{keyword}/{season}-Belgian-Pro-League-Stats'           
                gre_urls = f'https://fbref.com/en/comps/27/{season}/{keyword}/{season}-Super-League-Greece-Stats'           
                net_urls = f'https://fbref.com/en/comps/23/{season}/{keyword}/{season}-Eredivisie-Stats'
                pol_urls = f'https://fbref.com/en/comps/36/{season}/{keyword}/{season}-Ekstraklasa-Stats'           
                ser_urls = f'https://fbref.com/en/comps/54/{season}/{keyword}/{season}-Serbian-SuperLiga-Stats'
                tur_urls = f'https://fbref.com/en/comps/26/{season}/{keyword}/{season}-Super-Lig-Stats'


                dict = {spa_urls: [team_id,player_id], fra_urls: [team_id,player_id], eng_urls: [team_id,player_id], 
                        ger_urls: [team_id,player_id], por_urls: [team_id,player_id], ita_urls: [team_id,player_id], 
                        bul_urls: [team_id,player_id], aus_urls: [team_id,player_id], dan_urls: [team_id,player_id], 
                        bel_urls: [team_id,player_id], gre_urls: [team_id,player_id], net_urls: [team_id,player_id], 
                        pol_urls: [team_id,player_id], ser_urls: [team_id,player_id], tur_urls: [team_id,player_id]
                        }
                
                lst_of_dicts.append(dict)
                

    return lst_of_dicts

In [ ]:
# Web Scraping player data

# Set up Selenium options
options = Options()
options.headless = True  # Run in headless mode (no GUI)
service = Service(ChromeDriverManager().install())

csv_files = glob.glob(f'**.csv')

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

season_lst = ['2017-2018', '2018-2019', '2019-2020']

# Fetch the webpages for the desired seasons
for data_dict in league_stats(season_lst):
    
    for page, class_id in data_dict.items():
        for id in class_id:
            print(page, id)

        # Creating output csv name
            def create_filename(url, id):
                # Extract the relevant parts of the URL
                match = re.search(r'/(\d{4}-\d{4})/([^/]+)/(\d{4}-\d{4}-[^/]+)', url)
                table_id = re.search(r"'id': {'([^']*)'}", id)

                if match and id:
                    category = match.group(2).title()
                    stats = match.group(3).replace('-', '_')[:-6]
                    table = table_id.group(1)

                    if '_for' not in table:
                        table_type = 'Player'
                    else:
                        table_type = 'Team'

                    # Format the filename
                    filename = f"{stats}_{category}_{table_type}.csv"

                    return filename
                else:
                    return None

            csv_name = create_filename(page, str(id))

            if csv_name in csv_files:
                print('Dataset already exsits')

            else:
                driver.get(page)

                # Get the page source after JavaScript has rendered
                html = driver.page_source

                # Parse the HTML content with BeautifulSoup
                soup = BeautifulSoup(html, 'html.parser')

                # Find the table with the specified class
                table = soup.find('table', id)

                # Extract the data from the table
                if table:

                    # Find the thead element
                    thead = table.find('thead')
                    # Extract column headers with aria-label
                    headers = thead.find_all('th')
                    header_labels = [header.get('aria-label', header.text).strip() for header in headers if header.get('aria-label', header.text).strip()]
                            
                    # Use a set to remove duplicates, required due to multiple headers in the table across the web page
                    unique_header_labels = list(dict.fromkeys(header_labels))

                    # Removing unwanted blank fields scraped incorrectly
                    if 'Rk' in unique_header_labels:
                        filtered_lst = [header for header in header_labels if not (header.isdigit())][1:]
                    else:
                        filtered_lst = [header for header in header_labels if not (header.isdigit())]

                    if '/stats/' in page and 'npxG + xAG/90' not in filtered_lst:
                        if 'Matches' in filtered_lst:
                            filtered_lst.remove('Matches')
                            filtered_lst[-1]='npxG + xAG/90'
                            filtered_lst.append('Matches')

                        else:
                            filtered_lst[-1]='npxG + xAG/90'

                    rows = table.find_all('tr')
 
                    data = []

                    for row in rows:

                        columns = row.find_all('td')
                        
                        # Check if <th> and <a> tags exist
                        team_name_tag = row.find('th')
                        if team_name_tag and 'Team' in csv_name:

                            team_name_link = team_name_tag.find('a')
                            team_name = team_name_link.text if team_name_link else 'N/A'
                            row_data = [team_name] + [column.text for column in columns]
                            data.append(row_data)

                        else:
                            
                            row_data = [column.text for column in columns]
                            data.append(row_data)

                    # Write data to CSV
                    with open(csv_name, 'w', newline='', encoding='utf-8') as file:
                        
                        writer = csv.writer(file)
                        # Write header labels first
                        writer.writerow(filtered_lst)
                        # Write the rest of the data
                        writer.writerows(data)

                    print(f'Data has been scraped and saved to {csv_name}')

                else:
                    print("Table not found")

# Close the WebDriver
driver.quit()

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# Web Scraping Player market value 2024
mrkt_value_dict = {
    "https://www.transfermarkt.com/spieler-statistik/rekordmarktwerte/marktwertetop?position=Abwehr&land_id=0&plus=1": "def_transfer_values_raw.csv",
    "https://www.transfermarkt.com/spieler-statistik/rekordmarktwerte/marktwertetop?position=Mittelfeld&land_id=0&plus=1": "mid_transfer_values_raw.csv",
    "https://www.transfermarkt.com/spieler-statistik/rekordmarktwerte/marktwertetop?position=Sturm&land_id=0&plus=1": "atck_transfer_values_raw.csv",
    "https://www.transfermarkt.com/spieler-statistik/rekordmarktwerte/marktwertetop?position=Torwart&land_id=0&plus=1": "gkp_transfer_values_raw.csv"
}

for url, csv in mrkt_value_dict.items():
    # Base URL of the Transfermarkt page
    base_url = url
    # Headers to mimic a browser visit
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    # Function to scrape a single page
    def scrape_page(url):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", {"class": "items"})
        
        players = []
        ages = []
        dates = []
        record_values = []

        if table:
            for row in table.find_all("tr")[1:]:  # Skip the header row
                try:
                    player = row.select_one('td:nth-of-type(2) .hauptlink')
                    player_text = player.text.strip() if player else 'N/A'
                    if player_text in players:
                        continue  # Skip duplicates
                    players.append(player_text)
                except AttributeError:
                    players.append(None)

                try:
                    age = row.select_one('td:nth-of-type(5)')
                    age_text = age.text.strip() if age else 'N/A'
                    ages.append(age_text)
                except AttributeError:
                    ages.append(None)

                try:
                    date = row.select_one('td:nth-of-type(6)')
                    date_text = date.text.strip() if date else 'N/A'
                    dates.append(date_text)
                except AttributeError:
                    dates.append(None)

                try:
                    record_mv = row.select_one('td:nth-of-type(7)')
                    record_mv_text = record_mv.text.strip() if record_mv else 'N/A'
                    record_values.append(record_mv_text)
                except AttributeError:
                    record_values.append(None)

        return players, ages, dates, record_values

    # Initialize lists to store all players and values
    all_players = []
    all_ages = []
    all_dates = []
    all_record_values = []

    # Scrape the first page
    players, ages, dates, record_values = scrape_page(base_url)
    all_players.extend(players)
    all_ages.extend(ages)
    all_dates.extend(dates)
    all_record_values.extend(record_values)

    # Scrape up to 20 pages
    for page in range(2, 21):
        next_page_url = f"{base_url}&page={page}"
        players, ages, dates, record_values = scrape_page(next_page_url)

        if not players:  # If no players are found, break the loop
            break

        all_players.extend(players)
        all_ages.extend(ages)
        all_dates.extend(dates)
        all_record_values.extend(record_values)

    # Create a DataFrame to store the data
    df = pd.DataFrame({
        "Player": all_players,
        "Age": all_ages,
        "Date": all_dates,
        "Record MV": all_record_values
    })

    # Save the data to a CSV file
    df.to_csv(csv, index=False)
    print(f'Data has been scraped and saved to {csv}')


Data has been scraped and saved to def_transfer_values_raw.csv
Data has been scraped and saved to mid_transfer_values_raw.csv
Data has been scraped and saved to atck_transfer_values_raw.csv
Data has been scraped and saved to gkp_transfer_values_raw.csv


In [139]:
# Web Scraping Club Ranking Data

# Base URL
base_url = "https://www.transfermarkt.co.uk/statistik/klubrangliste"

# Headers to mimic a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

# Function to scrape a single page
def scrape_page(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find("table", {"class": "items"})
    
    data = []

    if table:
        # Extract data
        for row in table.find_all('tr', class_=['odd', 'even']):
            club_name = row.find('a', title=True)['title']
            country_name = row.find('img', class_='flaggenrahmen')['title']
            values = [td.text.strip() for td in row.find_all('td', class_='rechts')]
            overall = values.pop()
            data.append([club_name, country_name] + values + [overall])
    
    return data

all_teams = []

# Scrape the first page
data = scrape_page(base_url)
all_teams.extend(data)

# Scrape up to 20 pages
for page in range(2, 23):
    next_page_url = f"{base_url}?page={page}"
    data = scrape_page(next_page_url)
    all_teams.extend(data)

# Create DataFrame
df = pd.DataFrame(all_teams, columns=['Club Name', 'Country', '20/21', '21/22', '22/23', '23/24', '24/25', 'Overall'])

# Save to CSV
df.to_csv('club_rank_data.csv', index=False)


In [ ]:
# Web Scraping International Ranking Data

# URLs
url_2024 = 'https://www.transfermarkt.co.uk/statistik/weltrangliste/statistik/stat/plus/0/galerie/0?datum=2024-09-19'
url_2023 = 'https://www.transfermarkt.co.uk/statistik/weltrangliste/statistik/stat/plus/0/galerie/0?datum=2023-09-21'
url_2022 = 'https://www.transfermarkt.co.uk/statistik/weltrangliste/statistik/stat/plus/0/galerie/0?datum=2022-09-20'

url_lst = [url_2022, url_2023, url_2024]

# Headers to mimic a browser visit
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

for url in url_lst:

    dt = url[-10:]
    print(yr)
    # Function to scrape a single page
    def scrape_page(url):
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        table = soup.find("table", {"class": "items"})
        
        data = []

        if table:
            # Extract data
            for row in table.find_all('tr', class_=['odd', 'even']):
    
                rank = row.find('td', class_='zentriert cp').text.strip().split()[0]
                previous_rank = row.find('span', class_='icons_sprite')['title'].split(': ')[1]
                nation = row.find('a', title=True)['title']
                confederation = row.find_all('td', class_='zentriert')[1].text.strip()
                points = row.find('td', class_='zentriert hauptlink').text.strip()
    
                data.append([rank, previous_rank, nation, confederation, points])

        return data
    
    
    all_teams = []

    # Scrape the first page
    data = scrape_page(url)
    all_teams.extend(data)
    print(data)

   # Scrape up to 20 pages
    for num in range(2, 10):
        try:
            next_page_url = f"https://www.transfermarkt.co.uk/statistik/weltrangliste/statistik/stat/ajax/yw1/datum/{dt}/plus/0/galerie/0/page/{num}"
            data = scrape_page(next_page_url)
            all_teams.extend(data)
        except Exception as e:
            print(f"Error: Page Not Found")
            break

    print(all_teams)
    # Create DataFrame
    df = pd.DataFrame(all_teams, columns=['nation_rank','nation_prev_rank', 'nation', 'confederation', 'points'])

    # Save to CSV
    df.to_csv(f'nation_rank_data_{dt}_raw.csv', index=False)


In [ ]:
# Web Scraping Competition Data

dict_list = [
    {"url": "https://fbref.com/en/comps/season/2021-2022", 'id': "comps_intl_club_cup"},
    {"url": "https://fbref.com/en/comps/season/2021-2022", 'id': "comps_fa_club_cup"},
    {"url": "https://fbref.com/en/comps/season/2021-2022", 'id': "comps_1_fa_club_league_senior"},
    {"url": "https://fbref.com/en/comps/season/2022-2023", 'id': "comps_intl_club_cup"},
    {"url": "https://fbref.com/en/comps/season/2022-2023", 'id': "comps_fa_club_cup"},
    {"url": "https://fbref.com/en/comps/season/2022-2023", 'id': "comps_1_fa_club_league_senior"},
    {"url": "https://fbref.com/en/comps/season/2023-2024", 'id': "comps_intl_club_cup"},
    {"url": "https://fbref.com/en/comps/season/2023-2024", 'id': "comps_fa_club_cup"},
    {"url": "https://fbref.com/en/comps/season/2023-2024", 'id': "comps_1_fa_club_league_senior"}
]

# Set up Selenium options
options = Options()
options.headless = True  # Run in headless mode (no GUI)
service = Service(ChromeDriverManager().install())

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Fetch the webpages
for entry in dict_list:
    url = entry["url"]
    url_str = str(url)
    class_id = entry["id"]
    id = entry['id']
    driver.get(url)

    # Get the page source after JavaScript has rendered
    html = driver.page_source

    # Parse the HTML content with BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Find the table with the specified class
    table = soup.find('table', {'id': class_id})

    # Extract the data from the table
    if table:

        # Extract column headers with aria-label
        headers = table.find_all('th')
        header_labels = [header.get('aria-label') for header in headers if header.get('aria-label')]
        # header_labels =  list(set(header_labels))

        rows = table.find_all('tr')
        data = []
        
        for row in rows:
            # Extract the text from the <th> element
            league_name = row.find('th').text.strip()
            # Extract the text from the <td> elements
            columns = row.find_all('td')
            row_data = [league_name] + [column.text.strip() for column in columns]
            
            data.append(row_data)

        # Creating output csv name
        # Function to create the filename
        def create_filename(url, id):
            # Extract the season from the URL
            match = re.search(r'/(\d{4}-\d{4})', url)
            if match:
                season = match.group(1).replace('-', '_')
                # Format the filename
                filename = f"{id}_{season}_raw.csv"
                return filename
            else:
                return None
            
        csv_name = create_filename(url_str, id)
        # Write data to CSV
        with open(csv_name, 'w', newline='', encoding='utf-8') as file:
        
            writer = csv.writer(file)
            # Write header labels first
            writer.writerow(header_labels)
            # Write the rest of the data
            writer.writerows(data)

        print(f'Data has been scraped and saved to {csv_name}')

    else:
        print("Table not found")

# Close the WebDriver
driver.quit()

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL of the Wikipedia page
url = 'https://en.wikipedia.org/wiki/List_of_FIFA_country_codes'

# Send a request to fetch the HTML content
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables on the page
tables = soup.find_all('table', {'class': 'wikitable'})

# Initialize an empty DataFrame
final_df = pd.DataFrame()

# Loop through all tables and concatenate each to the final DataFrame
for table in tables:
    df = pd.read_html(str(table))[0]
    final_df = pd.concat([final_df, df], ignore_index=True)

# Save the final concatenated DataFrame to a CSV file
final_df.to_csv('raw_data/nation_raw_data/fifa_country_codes.csv', index=False)

print("All tables have been scraped, concatenated, and saved as a single CSV file.")


In [14]:
# Dictionary for the raw data, the key is the wbesite containing the raw data and the key is the ID relating to the table required on the web page

def league_table(seasons):

    lst_of_dicts = []
    dict = {}

    for season in seasons:
    
        spa_urls = f'https://fbref.com/en/comps/12/{season}/{season}-La-Liga-Stats'
        fra_urls = f'https://fbref.com/en/comps/13/{season}/{season}-Ligue-1-Stats'
        eng_urls = f'https://fbref.com/en/comps/9/{season}/{season}-Premier-League-Stats'
        ger_urls = f'https://fbref.com/en/comps/20/{season}/{season}-Bundesliga-Stats'
        por_urls = f'https://fbref.com/en/comps/32/{season}/{season}-Primeira-Liga-Stats'
        ita_urls = f'https://fbref.com/en/comps/11/{season}/{season}-Serie-A-Stats'
        bul_urls = f'https://fbref.com/en/comps/67/{season}/{season}-Bulgarian-First-League-Stats'
        aus_urls = f'https://fbref.com/en/comps/56/{season}/{season}-Austrian-Bundesliga-Stats'
        dan_urls = f'https://fbref.com/en/comps/50/{season}/{season}-Danish-Superliga-Stats'
        bel_urls = f'https://fbref.com/en/comps/37/{season}/{season}-Belgian-Pro-League-Stats'           
        gre_urls = f'https://fbref.com/en/comps/27/{season}/{season}-Super-League-Greece-Stats'           
        net_urls = f'https://fbref.com/en/comps/23/{season}/{season}-Eredivisie-Stats'
        pol_urls = f'https://fbref.com/en/comps/36/{season}/{season}-Ekstraklasa-Stats'           
        ser_urls = f'https://fbref.com/en/comps/54/{season}/{season}-Serbian-SuperLiga-Stats'
        tur_urls = f'https://fbref.com/en/comps/26/{season}/{season}-Super-Lig-Stats'

        url_lst = [ spa_urls, fra_urls, eng_urls, ger_urls, por_urls, ita_urls, bul_urls, aus_urls, dan_urls, bel_urls, gre_urls, net_urls, pol_urls, ser_urls, tur_urls ]

        for url in url_lst:
            match = re.search(r'comps/(\d+)/', url)
            match = match.group(1)

            team_league_id = {'id': {f'results{season}{match}1_overall'}}

            dict[url] = team_league_id
                
        lst_of_dicts.append(dict)
                

    return lst_of_dicts

In [ ]:
# Web Scraping player data

# Set up Selenium options
options = Options()
options.headless = True  # Run in headless mode (no GUI)
service = Service(ChromeDriverManager().install())

# Initialize the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Function to create a unique filename for each CSV
def create_filename(url):
    
    # Extract the relevant parts of the URL
    name = url.split('/')[-1].replace('-', '_') + "_table_placings.csv"
    return name

# Fetch the webpages for the desired seasons
for data_dict in league_table(['2021-2022', '2022-2023', '2023-2024']):
    
    for page, id in data_dict.items():

        print(page, id)

        csv_files = glob.glob(f'**.csv')

        csv_name = create_filename(page)

        if csv_name in csv_files:
            print('Dataset already exists')

        else:
            driver.get(page)

            # Get the page source after JavaScript has rendered
            html = driver.page_source

            # Parse the HTML content with BeautifulSoup
            soup = BeautifulSoup(html, 'html.parser')

            # Find the table with the specified class
            table = soup.find('table', id)

            # Extract the data from the table
            if table:

                # Find the thead element
                thead = table.find('thead')
                # Extract column headers with aria-label
                headers = thead.find_all('th')
                header_labels = [header.get('aria-label', header.text).strip() for header in headers if header.get('aria-label', header.text).strip()]
                        
                # Use a set to remove duplicates, required due to multiple headers in the table across the web page
                unique_header_labels = list(dict.fromkeys(header_labels))

                # Removing unwanted blank fields scraped incorrectly
                if 'Rank' in unique_header_labels:
                    filtered_lst = [header for header in header_labels if not (header.isdigit())][1:]
                else:
                    filtered_lst = [header for header in header_labels if not (header.isdigit())]

                rows = table.find_all('tr')

                data = []

                for row in rows:

                    columns = row.find_all('td')
                    
                    row_data = [column.text for column in columns]
                    data.append(row_data)

                # Write data to CSV
                with open(csv_name, 'w', newline='', encoding='utf-8') as file:
                    
                    writer = csv.writer(file)
                    # Write header labels first
                    writer.writerow(filtered_lst)
                    # Write the rest of the data
                    writer.writerows(data)

                print(f'Data has been scraped and saved to {csv_name}')

            else:
                print("Table not found")

# Close the WebDriver
driver.quit()
